In [39]:
import os
import cv2
import math   
import matplotlib.pyplot as plt
import pandas as pd  
import numpy as np  
from glob import glob
from tqdm import tqdm

In [40]:
train_labels = pd.read_csv("VGAF/txt_files/Train_labels.txt", sep = ' ') # train dataset
validation_labels = pd.read_csv("VGAF/txt_files/Val_labels.txt", sep = ' ') #validation dataset

# Train data and frames extraction

In [41]:
train_labels

,Vid_name,Label
0,2_1,2
1,2_2,3
2,2_3,1
3,2_4,2
4,3_1,3
...,...,...
2656,347_1,2
2657,347_2,3
2658,347_3,3
2659,347_4,3


In [42]:
video_train_dir = "VGAF/Train/"
train_video_names_with_path = [video_train_dir + x for x in os.listdir(video_train_dir)]

In [26]:
# #don't touch it until you need to recollect images
# for i in tqdm(range(train_labels.shape[0])):
#     count = 0
#     videoFile = train_labels['Vid_name'][i]
#     cap = cv2.VideoCapture("../course_work/VGAF/Train/" + train_video_names_with_path[i].split("/")[4])   # capturing the video from the given path
#     frameRate = cap.get(5) #frame rate
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder named Validation_Frames
#             filename = "../course_work/VGAF/Train_Frames/" + train_video_names_with_path[i].split("/")[4].split(".")[0] +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

In [63]:
import cv2 
import os
import numpy as np
from decord import VideoReader
from decord import cpu, gpu


def extract_frames(video_path, frames_dir, overwrite=False, start=-1, end=-1, frame_number=1):
    """
    Extract frames from a video using decord's VideoReader
    :param video_path: path of the video
    :param frames_dir: the directory to save the frames
    :param overwrite: to overwrite frames that already exist?
    :param start: start frame
    :param end: end frame
    :param frame_number: number of frames needed to be collected from video
    """   
    
    
    video_path = os.path.normpath(video_path)  # make the paths OS (Windows) compatible
    frames_dir = os.path.normpath(frames_dir)  # make the paths OS (Windows) compatible
    
    
    video_filename = video_path.split("/")[2].split(".")[0] # get file name

    assert os.path.exists(video_path)  # assert the video file exists   


    # load the VideoReader
    vr = VideoReader(video_path, ctx=cpu(0))  # can set to cpu or gpu .. ctx=gpu(0)
    needed = len(vr)/frame_number # frame spacing for collecting exact number of frames
                     
    if start < 0:  # if start isn't specified lets assume 0
        start = 1
    if end < 0:  # if end isn't specified assume the end of the video
        end = len(vr)

    frames_list = list(range(start, end, (int(needed)+1)))
    saved_count = 0

    if (int(needed)+1) > 25 and len(frames_list) < 1000:  # this is faster for every > 25 frames and can fit in memory
        frames = vr.get_batch(frames_list).asnumpy()
        for index, frame in zip(frames_list, frames):  # lets loop through the frames until the end
            save_path = os.path.join(frames_dir, "img_0000%d.jpg" % (index /(int(needed)+1)))  # create the save path
            if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                cv2.imwrite(save_path, cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))  # save the extracted image

    else:  # this is faster for every <25 and consumes small memory
        for index in range(start, end):  # lets loop through the frames until the end
            frame = vr[index]  # read an image from the capture            
            if index % (int(needed)+1) == 0:  # if this is a frame we want to write out based on the 'every' argument
                save_path = os.path.join(frames_dir,  "img_0000%d.jpg" % (index /(int(needed)+1)))  # create the save path
                if not os.path.exists(save_path) or overwrite:  # if it doesn't exist or we want to overwrite anyways
                    cv2.imwrite(save_path, cv2.cvtColor(frame.asnumpy(), cv2.COLOR_RGB2BGR))  # save the extracted image

In [64]:
# frames_dir = r"/media/nitro/hdd/course_work/datasets/Kinetics400/frames/"
# for i, path in tqdm(enumerate(train_video_names_with_path)):
#     frames_dir_vid = os.path.join(frames_dir + "video_" + train_video_names_with_path[i].split("/")[2].split(".")[0])
#     os.mkdir(frames_dir_vid)
#     extract_frames(video_path=path, frames_dir= frames_dir_vid, overwrite=True, frame_number=8)

2661it [11:31,  3.85it/s]


In [43]:
# images = glob("VGAF/Train_Frames/*.jpg")
# train_images = []
# train_images_labels = []
# train_images_clean = []
# for i in tqdm(range(len(images))):
#     train_images.append(images[i].split("/")[2])
#     train_images_clean.append(images[i].split("/")[2].split("_")[0] + "_" + images[i].split("/")[2].split("_")[1])
# train_data = pd.DataFrame()
# train_data['image'] = train_images
# train_data["image_clean"] = train_images_clean
# train_data.insert(2, "label", 0)

100%|██████████| 21288/21288 [00:00<00:00, 701467.09it/s]


In [46]:
train_data = train_labels.copy(deep=True)
train_data.insert(0, "path", "sadad")
train_data.insert(1, "num_frames", 8)
train_data

,path,num_frames,Vid_name,Label
0,sadad,8,2_1,2
1,sadad,8,2_2,3
2,sadad,8,2_3,1
3,sadad,8,2_4,2
4,sadad,8,3_1,3
...,...,...,...,...
2656,sadad,8,347_1,2
2657,sadad,8,347_2,3
2658,sadad,8,347_3,3
2659,sadad,8,347_4,3


In [47]:
train_data.head()

,path,num_frames,Vid_name,Label
0,sadad,8,2_1,2
1,sadad,8,2_2,3
2,sadad,8,2_3,1
3,sadad,8,2_4,2
4,sadad,8,3_1,3


In [45]:
# for i in tqdm(train_labels["Vid_name"]):
#     for a in train_data["image_clean"]:
#         if i == a:
#             vid_name_index = train_labels[train_labels["Vid_name"]==i].index.values[0]
#             temp = train_labels.at[vid_name_index, "Label"]
#             for j in train_data[train_data["image_clean"]==i].index.values:
#                 train_data.at[j, "label"] = temp

100%|██████████| 2661/2661 [00:40<00:00, 65.43it/s]


In [48]:
for i in tqdm(train_labels["Vid_name"]):
    for a in train_data["Vid_name"]:
        if i == a: 
            vid_name_index = train_labels[train_labels["Vid_name"]==i].index.values[0]
            temp = train_labels.at[vid_name_index, "Vid_name"]
            for j in train_data[train_data["Vid_name"]==i].index.values:
#                 train_data.at[j, "path"] = "frames/" + temp
                train_data.at[j, "path"] = "frames/video_" + temp

100%|██████████| 2661/2661 [00:02<00:00, 887.86it/s]


In [49]:
train_data.head(40)

,path,num_frames,Vid_name,Label
0,frames/video_2_1,8,2_1,2
1,frames/video_2_2,8,2_2,3
2,frames/video_2_3,8,2_3,1
3,frames/video_2_4,8,2_4,2
4,frames/video_3_1,8,3_1,3
5,frames/video_3_2,8,3_2,1
6,frames/video_3_3,8,3_3,3
7,frames/video_3_4,8,3_4,2
8,frames/video_3_5,8,3_5,2
9,frames/video_3_6,8,3_6,3


In [49]:
# train_data.drop("image_clean", axis=1, inplace=True)

In [50]:
train_data.to_csv('VGAF/txt_files/train_data.txt',header=True, index=False)

In [51]:
pd.read_csv("VGAF/txt_files/train_data.csv")

,path,num_frames,Vid_name,Label
0,frames/video_2_1,8,2_1,2
1,frames/video_2_2,8,2_2,3
2,frames/video_2_3,8,2_3,1
3,frames/video_2_4,8,2_4,2
4,frames/video_3_1,8,3_1,3
...,...,...,...,...
2656,frames/video_347_1,8,347_1,2
2657,frames/video_347_2,8,347_2,3
2658,frames/video_347_3,8,347_3,3
2659,frames/video_347_4,8,347_4,3


# VALIDATION DATA AND FRAMES EXTRACTION

In [52]:
video_validation_dir = "VGAF/Val/"
validation_video_names_with_path = [video_validation_dir + x for x in os.listdir(video_validation_dir)]

In [53]:
validation_labels

,Vid_name,Label
0,1_1,3
1,1_2,3
2,5_1,1
3,5_2,1
4,5_3,1
...,...,...
761,335_8,2
762,335_9,1
763,335_10,2
764,342_1,3


In [176]:
# #don't touch it until you need to recollect images
# for i in tqdm(range(validation_labels.shape[0])):
#     count = 0
#     videoFile = validation_labels['Vid_name'][i]
#     cap = cv2.VideoCapture("../course_work/VGAF/Val/" + validation_video_names_with_path[i].split("/")[4])   # capturing the video from the given path
#     frameRate = cap.get(5) #frame rate
#     while(cap.isOpened()):
#         frameId = cap.get(1) #current frame number
#         ret, frame = cap.read()
#         if (ret != True):
#             break
#         if (frameId % math.floor(frameRate) == 0):
#             # storing the frames in a new folder named Validation_Frames
#             filename = "../course_work/VGAF/Validation_Frames/" +validation_video_names_with_path[i].split("/")[4].split(".")[0] +"_frame%d.jpg" % count;count+=1
#             cv2.imwrite(filename, frame)
#     cap.release()

In [65]:
# Script i use for 24.01.2021

frames_dir = r"/media/nitro/hdd/course_work/datasets/Kinetics400/frames/"
for i, path in tqdm(enumerate(validation_video_names_with_path)):
    frames_dir_vid = os.path.join(frames_dir + "video_" +  validation_video_names_with_path[i].split("/")[2].split(".")[0])
    os.mkdir(frames_dir_vid)
    extract_frames(video_path=path, frames_dir= frames_dir_vid, overwrite=True, frame_number=8)

766it [03:05,  4.12it/s]


In [178]:
# validation_video_names_with_path[1].split("/")[4]

In [54]:
validation_labels

,Vid_name,Label
0,1_1,3
1,1_2,3
2,5_1,1
3,5_2,1
4,5_3,1
...,...,...
761,335_8,2
762,335_9,1
763,335_10,2
764,342_1,3


In [55]:
val_data = validation_labels.copy(deep=True)
val_data.insert(0, "path", "sadad")
val_data.insert(1, "num_frames", 8)
val_data

,path,num_frames,Vid_name,Label
0,sadad,8,1_1,3
1,sadad,8,1_2,3
2,sadad,8,5_1,1
3,sadad,8,5_2,1
4,sadad,8,5_3,1
...,...,...,...,...
761,sadad,8,335_8,2
762,sadad,8,335_9,1
763,sadad,8,335_10,2
764,sadad,8,342_1,3


In [182]:
# images = glob("../course_work/VGAF/Validation_Frames/*.jpg")
# val_images = []
# val_images_labels = []
# val_images_clean = []
# for i in tqdm(range(len(images))):
#     val_images.append(images[i].split("/")[4])
#     val_images_clean.append(images[i].split("/")[4].split("_")[0] + "_" + images[i].split("/")[4].split("_")[1])
# val_data = pd.DataFrame()
# val_data['image'] = val_images
# val_data["image_clean"] = val_images_clean
# val_data.insert(2, "label", 0)

In [16]:
val_data

,path,num_frames,Vid_name,Label
0,sadad,8,1_1,3
1,sadad,8,1_2,3
2,sadad,8,5_1,1
3,sadad,8,5_2,1
4,sadad,8,5_3,1
...,...,...,...,...
761,sadad,8,335_8,2
762,sadad,8,335_9,1
763,sadad,8,335_10,2
764,sadad,8,342_1,3


In [184]:
# script for labels

# for i in tqdm(validation_labels["Vid_name"]):
#     for a in val_data["image_clean"]:
#         if i == a: 
#             vid_name_index = validation_labels[validation_labels["Vid_name"]==i].index.values[0]
#             temp = validation_labels.at[vid_name_index, "Label"]
#             for j in val_data[val_data["image_clean"]==i].index.values:
#                 val_data.at[j, "label"] = temp

In [56]:
for i in tqdm(validation_labels["Vid_name"]):
    for a in val_data["Vid_name"]:
        if i == a: 
            vid_name_index = validation_labels[validation_labels["Vid_name"]==i].index.values[0]
            temp = validation_labels.at[vid_name_index, "Vid_name"]
            for j in val_data[val_data["Vid_name"]==i].index.values:
#                 val_data.at[j, "path"] = "frames/" + temp
                val_data.at[j, "path"] = "frames/video_" + temp

100%|██████████| 766/766 [00:00<00:00, 1396.77it/s]


In [57]:
val_data

,path,num_frames,Vid_name,Label
0,frames/video_1_1,8,1_1,3
1,frames/video_1_2,8,1_2,3
2,frames/video_5_1,8,5_1,1
3,frames/video_5_2,8,5_2,1
4,frames/video_5_3,8,5_3,1
...,...,...,...,...
761,frames/video_335_8,8,335_8,2
762,frames/video_335_9,8,335_9,1
763,frames/video_335_10,8,335_10,2
764,frames/video_342_1,8,342_1,3


In [188]:
# val_data.drop("image_clean", axis=1,inplace = True)

In [58]:
val_data.to_csv('../course_work/VGAF/txt_files/val_data.txt',header=True, index=False)

In [59]:
pd.read_csv("../course_work/VGAF/txt_files/val_data.csv")

,path,num_frames,Vid_name,Label
0,frames/video_1_1,8,1_1,3
1,frames/video_1_2,8,1_2,3
2,frames/video_5_1,8,5_1,1
3,frames/video_5_2,8,5_2,1
4,frames/video_5_3,8,5_3,1
...,...,...,...,...
761,frames/video_335_8,8,335_8,2
762,frames/video_335_9,8,335_9,1
763,frames/video_335_10,8,335_10,2
764,frames/video_342_1,8,342_1,3


# AUDIO TRAIN EXTRACTION

In [25]:
import moviepy.editor as mp

In [26]:
# %%capture
# # Don't touch until you need to recollect train audio
# for i in range(train_labels.shape[0]):
#     videoFile = train_labels['Vid_name'][i]
#     video = mp.VideoFileClip("../course_work/VGAF/Train/" + videoFile + ".mp4")
#     filename = "../course_work/VGAF/Train_Audio/" + train_labels['Vid_name'][i] +".wav"
#     video.audio.write_audiofile(filename)

In [27]:
audio_train_dir = "../course_work/VGAF/Train/"
train_audio_names_with_path = [audio_train_dir + x for x in os.listdir(audio_train_dir)]

In [28]:
train_audio_names_with_path[1].split("/")[4]

'328_19.mp4'

In [19]:
audio = glob("../course_work/VGAF/Train_Audio/*.wav")
train_audio = []
train_audio_labels = []
train_audio_clean = []
for i in tqdm(range(len(audio))):
    train_audio.append(audio[i].split("/")[4])
    train_audio_clean.append(audio[i].split("/")[4].split(".")[0])
train_data_audio = pd.DataFrame()
train_data_audio['audio'] = train_audio
train_data_audio["audio_clean"] = train_audio_clean
train_data_audio.insert(2, "label", 0)

100%|██████████| 2661/2661 [00:00<00:00, 1241357.24it/s]


In [20]:
audio = glob("../course_work/VGAF/Train_Audio/*.wav")
audio[1].split("/")[4].split(".")[0]

'155_13'

In [21]:
train_data_audio

,audio,audio_clean,label
0,13_2.wav,13_2,0
1,155_13.wav,155_13,0
2,203_11.wav,203_11,0
3,6_51.wav,6_51,0
4,72_11.wav,72_11,0
...,...,...,...
2656,281_8.wav,281_8,0
2657,245_4.wav,245_4,0
2658,340_1.wav,340_1,0
2659,180_11.wav,180_11,0


In [22]:
for i in tqdm(train_labels["Vid_name"]):
    for a in train_data_audio["audio_clean"]:
        if i == a: 
            audio_name_index = train_labels[train_labels["Vid_name"]==i].index.values[0]
            temp = train_labels.at[audio_name_index, "Label"]
            for j in train_data_audio[train_data_audio["audio_clean"]==i].index.values:
                train_data_audio.at[j, "label"] = temp

100%|██████████| 2661/2661 [00:02<00:00, 923.14it/s]


In [23]:
train_data_audio

,audio,audio_clean,label
0,13_2.wav,13_2,2
1,155_13.wav,155_13,2
2,203_11.wav,203_11,1
3,6_51.wav,6_51,2
4,72_11.wav,72_11,1
...,...,...,...
2656,281_8.wav,281_8,1
2657,245_4.wav,245_4,1
2658,340_1.wav,340_1,3
2659,180_11.wav,180_11,1


In [24]:
train_data_audio.drop("audio_clean", axis=1,inplace = True)

In [25]:
train_data_audio.to_csv('../course_work/VGAF/txt_files/train_data_audio.csv',header=True, index=False)

In [26]:
pd.read_csv('../course_work/VGAF/txt_files/train_data_audio.csv')

,audio,label
0,13_2.wav,2
1,155_13.wav,2
2,203_11.wav,1
3,6_51.wav,2
4,72_11.wav,1
...,...,...
2656,281_8.wav,1
2657,245_4.wav,1
2658,340_1.wav,3
2659,180_11.wav,1


# AUDIO VALIDATION EXTRACTION

In [37]:
audio_validation_dir = "../course_work/VGAF/Val/"
validation_audio_names_with_path = [audio_validation_dir + x for x in os.listdir(audio_validation_dir)]

In [38]:
# %%capture
# # Don't touch until you need to recollect train audio
# for i in range(validation_labels.shape[0]):
#     videoFile = validation_labels['Vid_name'][i]
#     video = mp.VideoFileClip("../course_work/VGAF/Val/" + videoFile + ".mp4")
#     filename = "../course_work/VGAF/Validation_Audio/" + validation_labels['Vid_name'][i] +".wav"
#     video.audio.write_audiofile(filename)

In [27]:
audio = glob("../course_work/VGAF/Validation_Audio/*.wav")
validation_audio = []
validation_audio_labels = []
validation_audio_clean = []
for i in tqdm(range(len(audio))):
    validation_audio.append(audio[i].split("/")[4])
    validation_audio_clean.append(audio[i].split("/")[4].split(".")[0])
validation_data_audio = pd.DataFrame()
validation_data_audio['audio'] = validation_audio
validation_data_audio["audio_clean"] = validation_audio_clean
validation_data_audio.insert(2, "label", 0)

100%|██████████| 766/766 [00:00<00:00, 892206.85it/s]


In [28]:
validation_data_audio

,audio,audio_clean,label
0,294_22.wav,294_22,0
1,181_5.wav,181_5,0
2,298_12.wav,298_12,0
3,289_51.wav,289_51,0
4,325_46.wav,325_46,0
...,...,...,...
761,298_24.wav,298_24,0
762,325_16.wav,325_16,0
763,294_46.wav,294_46,0
764,86_10.wav,86_10,0


In [29]:
for i in tqdm(validation_labels["Vid_name"]):
    for a in validation_data_audio["audio_clean"]:
        if i == a: 
            audio_name_index = validation_labels[validation_labels["Vid_name"]==i].index.values[0]
            temp = validation_labels.at[audio_name_index, "Label"]
            for j in validation_data_audio[validation_data_audio["audio_clean"]==i].index.values:
                validation_data_audio.at[j, "label"] = temp

100%|██████████| 766/766 [00:00<00:00, 1400.89it/s]


In [30]:
validation_data_audio

,audio,audio_clean,label
0,294_22.wav,294_22,2
1,181_5.wav,181_5,1
2,298_12.wav,298_12,1
3,289_51.wav,289_51,1
4,325_46.wav,325_46,3
...,...,...,...
761,298_24.wav,298_24,2
762,325_16.wav,325_16,3
763,294_46.wav,294_46,1
764,86_10.wav,86_10,1


In [31]:
validation_data_audio.drop("audio_clean", axis=1,inplace = True)

In [32]:
validation_data_audio.to_csv('../course_work/VGAF/txt_files/validation_data_audio.csv',header=True, index=False)

In [33]:
pd.read_csv('../course_work/VGAF/txt_files/validation_data_audio.csv')

,audio,label
0,294_22.wav,2
1,181_5.wav,1
2,298_12.wav,1
3,289_51.wav,1
4,325_46.wav,3
...,...,...
761,298_24.wav,2
762,325_16.wav,3
763,294_46.wav,1
764,86_10.wav,1
